In [43]:
import json
import pandas as pd

In [2]:
fname = '../data-raw/Most+Recent+Cohorts+(All+Data+Elements).csv'
df = pd.read_csv(fname, na_values=['NULL', 'PrivacySuppressed'])

print df.shape
print df.iloc[0]

(7804, 1728)
UNITID                                                        100654
OPEID                                                         100200
opeid6                                                          1002
INSTNM                                      Alabama A & M University
CITY                                                          Normal
STABBR                                                            AL
ZIP                                                            35762
AccredAgency       Southern Association of Colleges and Schools C...
INSTURL                                                www.aamu.edu/
NPCURL                galileo.aamu.edu/netpricecalculator/npcalc.htm
HCM2                                                               0
main                                                               1
NUMBRANCH                                                          1
PREDDEG                                                            3
HIGHDEG              

In [3]:
# Predominantly bachelor's degree granting (PREDDEG = 3)
# No Special Focus Institutions (CCBASIC < 24)
# 4-year schools, all sizes (CCSIZSET >= 6)

dg = df.copy()
print len(dg)

dg = dg[dg.PREDDEG == 3]
print len(dg)

dg = dg[dg.CCBASIC < 24]
print len(dg)

dg = dg[dg.CCSIZSET >= 6]
print len(dg)

7804
2133
1614
1608


In [4]:
dg.iloc[0]

UNITID                                                        100654
OPEID                                                         100200
opeid6                                                          1002
INSTNM                                      Alabama A & M University
CITY                                                          Normal
STABBR                                                            AL
ZIP                                                            35762
AccredAgency       Southern Association of Colleges and Schools C...
INSTURL                                                www.aamu.edu/
NPCURL                galileo.aamu.edu/netpricecalculator/npcalc.htm
HCM2                                                               0
main                                                               1
NUMBRANCH                                                          1
PREDDEG                                                            3
HIGHDEG                           

In [5]:
# % Who Graduate In 6 years = C150_4_POOLED
# % Of Students Who Are First Generation College Students = PAR_ED_PCT_1STGEN
# % Of Students Recieving Federal Loans = PCTFLOAN
# % Of Students Recieving Pell Grants = PCTPELL

# Default Rate = CDR3

# Average Wage, 6 Years After Entry = md_earn_wne_p6 (median), mn_earn_wne_p6 (mean), sd_earn_wne_p6 (sd)
# Median Wage, 10 years After Entry = md_earn_wne_p10 (median), mn_earn_wne_p10 (mean), sd_earn_wne_p10 (sd)

# Average Net Price = COALESCE(NPT4_PUB, NPT4_PRIV)
# Net Price For Students Whose Families Earn less than $48,000 = COALESCE(NPT4_048_PUB, NPT4_048_PRIV)

In [30]:
dh = dg.copy()

cols_to_keep = [
    'UNITID', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'INSTURL',
    'C150_4_POOLED', 'PAR_ED_PCT_1STGEN', 'PCTFLOAN', 'PCTPELL', 'CDR3',
    'md_earn_wne_p6', 'mn_earn_wne_p6', 'sd_earn_wne_p6',
    'md_earn_wne_p10', 'mn_earn_wne_p10', 'sd_earn_wne_p10',
    'NPT4_PUB', 'NPT4_PRIV', 'NPT4_048_PUB', 'NPT4_048_PRIV',
]

dh = dh[cols_to_keep]

# uppercase all column names
dh.columns = [c.upper() for c in dh.columns]

dh["NPT4_PUB_PRIV"] = dh[["NPT4_PUB", "NPT4_PRIV"]].sum(axis=1)
dh["NPT4_048_PUB_PRIV"] = dh[["NPT4_048_PUB", "NPT4_048_PRIV"]].sum(axis=1)
dh.drop(['NPT4_PUB', 'NPT4_PRIV', 'NPT4_048_PUB', 'NPT4_048_PRIV'],inplace=True,axis=1)

dh.iloc[0]

UNITID                                 100654
INSTNM               Alabama A & M University
CITY                                   Normal
STABBR                                     AL
ZIP                                     35762
INSTURL                         www.aamu.edu/
C150_4_POOLED                        0.308718
PAR_ED_PCT_1STGEN                    0.389902
PCTFLOAN                               0.8204
PCTPELL                                0.7115
CDR3                                     16.3
MD_EARN_WNE_P6                          22800
MN_EARN_WNE_P6                          26100
SD_EARN_WNE_P6                          21100
MD_EARN_WNE_P10                         31400
MN_EARN_WNE_P10                         35300
SD_EARN_WNE_P10                         27800
NPT4_PUB_PRIV                           13415
NPT4_048_PUB_PRIV                       12807
Name: 0, dtype: object

In [32]:
# summary stats of numeric cols

numeric_cols = [
    'C150_4_POOLED', 'PAR_ED_PCT_1STGEN', 'PCTFLOAN', 'PCTPELL', 'CDR3',
     'MD_EARN_WNE_P6', 'MN_EARN_WNE_P6', 'SD_EARN_WNE_P6',
     'MD_EARN_WNE_P10', 'MN_EARN_WNE_P10', 'SD_EARN_WNE_P10',
     'NPT4_PUB_PRIV', 'NPT4_048_PUB_PRIV'
]

# get summary stats
dh_ss = dh[numeric_cols].describe()

# limit to count, mean, and standard deviation
dh_ss = dh_ss.loc['count':'std']

# convert to json
stats = json.loads(dh_ss.to_json())

print stats

{u'PAR_ED_PCT_1STGEN': {u'count': 1575.0, u'std': 0.1127222318, u'mean': 0.3496813616}, u'SD_EARN_WNE_P6': {u'count': 1574.0, u'std': 9470.1659943501, u'mean': 25264.9936467598}, u'C150_4_POOLED': {u'count': 1559.0, u'std': 0.1949804698, u'mean': 0.5147736627}, u'CDR3': {u'count': 1604.0, u'std': 5.9799661138, u'mean': 8.7354738155}, u'MN_EARN_WNE_P10': {u'count': 1562.0, u'std': 12688.5065512873, u'mean': 47231.1139564661}, u'MD_EARN_WNE_P6': {u'count': 1574.0, u'std': 8037.1751025618, u'mean': 33117.1537484117}, u'MN_EARN_WNE_P6': {u'count': 1574.0, u'std': 9312.2515791672, u'mean': 36346.9504447268}, u'PCTPELL': {u'count': 1607.0, u'std': 0.1752264466, u'mean': 0.3972315495}, u'MD_EARN_WNE_P10': {u'count': 1562.0, u'std': 10037.4164791538, u'mean': 42218.6299615877}, u'PCTFLOAN': {u'count': 1607.0, u'std': 0.1770724115, u'mean': 0.6014067828}, u'NPT4_048_PUB_PRIV': {u'count': 1570.0, u'std': 5812.1177095323, u'mean': 15045.1242038217}, u'NPT4_PUB_PRIV': {u'count': 1570.0, u'std': 66

In [33]:
# add normalized z-score for each numeric col

for c in numeric_cols:
    dh['{}_z'.format(c)] = dh[c].apply(lambda x: 0 if pd.isnull(x) else (x - stats[c]['mean']) / stats[c]['std'])

dh.iloc[0]

UNITID                                   100654
INSTNM                 Alabama A & M University
CITY                                     Normal
STABBR                                       AL
ZIP                                       35762
INSTURL                           www.aamu.edu/
C150_4_POOLED                          0.308718
PAR_ED_PCT_1STGEN                      0.389902
PCTFLOAN                                 0.8204
PCTPELL                                  0.7115
CDR3                                       16.3
MD_EARN_WNE_P6                            22800
MN_EARN_WNE_P6                            26100
SD_EARN_WNE_P6                            21100
MD_EARN_WNE_P10                           31400
MN_EARN_WNE_P10                           35300
SD_EARN_WNE_P10                           27800
NPT4_PUB_PRIV                             13415
NPT4_048_PUB_PRIV                         12807
C150_4_POOLED_z                         -1.0568
PAR_ED_PCT_1STGEN_z                     

In [45]:
# output to json

dh.to_json('../data-clean/college-data.json', orient='records')